# Regression problem with Deep Learning (Keras)


In this task, we are going to build a neural network which will predict the house of a price, based on some relevant parameters.

At a high-level, neural networks are either encoders, decoders, or a combination of both. Encoders find patterns in raw data to form compact, useful representations. Decoders generate new data or high-resolution useful infomation from those representations. 

As a reminder, **Feed Forward Neural Networks (FFNNs)** is a network which its goal is to work on classification and regression problems, based on features (X->Y mapping, supervised problem). See the [aspect](#Part-1:-Boston-Housing-Price-Prediction-with-Feed-Forward-Neural-Networks) of this notebook for an example.

## Part 0: Prerequisites:


[tf.keras](https://www.tensorflow.org/guide/keras) is the simplest way to build and train neural network models in TensorFlow (w/ Keras).

Note that there's [tf.keras](https://www.tensorflow.org/guide/keras) (comes with TensorFlow) and there's [Keras](https://keras.io/) (standalone). You should be using [tf.keras](https://www.tensorflow.org/guide/keras) because (1) it comes with TensorFlow so you don't need to install anything extra and (2) it comes with powerful TensorFlow-specific features.

In [1]:
# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Conv1D, MaxPooling2D, Dropout, Flatten, Dense

# Commonly used modules
import numpy as np
import os
import sys

# Images, plots, display, and visualization
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

print(tf.__version__)

2022-03-13 16:38:25.183895: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-03-13 16:38:25.183937: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


2.7.0


## Part 1: Boston Housing Price Prediction with Feed Forward Neural Networks

Let's start with using a fully-connected neural network to do predict housing prices. The following image highlights the difference between regression and classification (see part 2). Given an observation as input, **regression** outputs a continuous value (e.g., exact temperature) and classificaiton outputs a class/category that the observation belongs to.

<img src="https://i.imgur.com/vvSoAzg.jpg" alt="classification_regression" width="400"/>

For the Boston housing dataset, we get 506 rows of data, with 13 features in each. Our task is to build a regression model that takes these 13 features as input and output a single value prediction of the "median value of owner-occupied homes (in thousands of USD)."

Now, we load the dataset. Loading the dataset returns four NumPy arrays:

* The `train_images` and `train_labels` arrays are the *training set*—the data the model uses to learn.
* The model is tested against the *test set*, the `test_images`, and `test_labels` arrays.

In [2]:
(train_features, train_labels), (test_features, test_labels) = keras.datasets.boston_housing.load_data()

# get per-feature statistics (mean, standard deviation) from the training set to normalize by
train_mean = np.mean(train_features, axis=0)
train_std = np.std(train_features, axis=0)
train_features = (train_features - train_mean) / train_std
print("Train dataset size: X: ", train_features.shape)
print("Train dataset size: y: ", train_labels.shape)

print("Test dataset size: X: ", test_features.shape)
print("Test dataset size: y: ", test_labels.shape)

Train dataset size: X:  (404, 13)
Train dataset size: y:  (404,)
Test dataset size: X:  (102, 13)
Test dataset size: y:  (102,)


In [3]:
train_features

array([[-0.27224633, -0.48361547, -0.43576161, ...,  1.14850044,
         0.44807713,  0.8252202 ],
       [-0.40342651,  2.99178419, -1.33391162, ..., -1.71818909,
         0.43190599, -1.32920239],
       [ 0.1249402 , -0.48361547,  1.0283258 , ...,  0.78447637,
         0.22061726, -1.30850006],
       ...,
       [-0.40202987,  0.99079651, -0.7415148 , ..., -0.71712291,
         0.07943894, -0.67776904],
       [-0.17292018, -0.48361547,  1.24588095, ..., -1.71818909,
        -0.98764362,  0.42083466],
       [-0.40422614,  2.04394792, -1.20161456, ..., -1.30866202,
         0.23317118, -1.15392266]])

### Build the model

Building the neural network requires configuring the layers of the model, then compiling the model. First we stack a few layers together using `keras.Sequential`. <br>
Next we configure the loss function, optimizer, and metrics to monitor. These are added during the model's compile step:

* *Loss function* - measures how accurate the model is during training, we want to minimize this with the optimizer.
* *Optimizer* - how the model is updated based on the data it sees and its loss function.
* *Metrics* - used to monitor the training and testing steps.

Your goal is to build a network that is getting input as the input size, has 3 hidden layers (each one with a different size), and one output layer (as we want to have 1 number at the end).
Also, loss function should be: mse (mean squared error), optimizer should be Adam, and we should use mae + mse as our metrics.

In [8]:
def build_model():
    model = keras.Sequential([
        Dense(20, activation=tf.nn.relu, input_shape=[len(train_features[0])]),
        Dense(20, activation=tf.nn.relu, input_shape=[20]),
        Dense(20, activation=tf.nn.relu, input_shape=[20]),
        Dense(1)
    ])

    model.compile(optimizer=tf.optimizers.Adam(), 
                  loss='mse',
                  metrics=['mae', 'mse'])
    return model

def build_cnn_model():
    model = keras.Sequential()
    model.add(Conv1D(32, (2), activation='relu', input_shape=[len(train_features[0])]))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv1D(64, (2), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv1D(64, (2), activation='relu')),
    model.add(Dense(1))
    
    model = keras.Sequential(optimizer = 'adam',
                             loss=tf.keras.losses.MeanSquaredError(reduction="auto", name="mean_squared_error"), metrics=['mae', 'mse'])
    
    return model

### Train the model

Training the neural network model requires the following steps:

1. Feed the training data to the model—in this example, the `train_features` and `train_labels` arrays.
2. The model learns to associate features and labels.
3. We ask the model to make predictions about a test set—in this example, the `test_features` array. We verify that the predictions match the labels from the `test_labels` array. 

To start training,  call the `model.fit` method—the model is "fit" to the training data:

In [7]:
# this helps makes our output less verbose but still shows progress
# Here and above you have a full example of a NN model performance plotting
# You should replicate this mechanism and apply it to other models as well (see TODO below)
class PrintDot(keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs):
        if epoch % 100 == 0: print('')
        print('.', end='')

# model instantiation        
model = build_cnn_model()

# TODO: Read about EarlyStoppingRound!
early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=50)

# training the model and storing data into history
history = model.fit(train_features, train_labels, epochs=1000, verbose=0, validation_split = 0.1,
                    callbacks=[early_stop, PrintDot()])

# using Pandas, generating data
hist = pd.DataFrame(history.history)
# Adding a column called epoch to that DataFrame
hist['epoch'] = history.epoch
hist


ValueError: Input 0 of layer "conv1d_1" is incompatible with the layer: expected min_ndim=3, found ndim=2. Full shape received: (None, 13)

In [ ]:

# RMSE 
### TODO: Calculate RMSE by the val_mse column
rmse_final = np.sqrt((hist.val_mse - hist.mse) ** 2) / len(hist)

In [ ]:
print(rmse_final)
rmse_final = np.sum(rmse_final)
print(rmse_final)

In [ ]:

print()
print('Final Mean Square Error on validation set: {}'.format(round(rmse_final, 3)))

Now, let's plot the loss function measure on the training and validation sets. The validation set is used to prevent overfitting ([learn more about it here](https://www.tensorflow.org/tutorials/keras/overfit_and_underfit)). However, because our network is small, the training convergence without noticeably overfitting the data as the plot shows.

In [ ]:
# Here and above you have a full example of a NN model performance plotting
# You should replicate this mechanism and apply it to other models as well (see TODO below)
def plot_history():
    plt.figure()
    plt.xlabel('Epoch')
    plt.ylabel('Mean Square Error [Thousand Dollars$^2$]')
    plt.plot(hist['epoch'], hist['mse'], label='Train Error')
    plt.plot(hist['epoch'], hist['val_mse'], label = 'Val Error')
    plt.legend()
    plt.ylim([0,50])

plot_history()


### TODO: Make sure to have the same flow for CNN as well, as the goal will be to see 2 plots:
# each one contain train error and val error on each model (NN, CNN) respectively

Next, compare how the model performs on the test dataset:

In [ ]:
# No need to touch
test_features_norm = (test_features - train_mean) / train_std
mse, _, _ = model.evaluate(test_features_norm, test_labels)
rmse = np.sqrt(mse)
print('Root Mean Square Error on test set: {}'.format(round(rmse, 3)))

## TODO: Make sure to have a mechanism for CNN as well, as above

## Acknowledgements

The contents of this tutorial is based on and inspired by the work of [TensorFlow team](https://www.tensorflow.org) (see their [Colab notebooks](https://www.tensorflow.org/tutorials/)), our [MIT Human-Centered AI team](https://hcai.mit.edu), and individual pieces referenced in the [MIT Deep Learning](https://deeplearning.mit.edu) course slides.

### Done! You just finished a competition between NN and CNN, where both network knows how to predict house prices